In [ ]:
import pandas as pd

df = pd.read_csv('/content/combined_newcsv.csv', skipinitialspace=True)
df = df.drop(['Unnamed: 0'], axis=1)
df

,Review Rate,Review Text
0,5 stars,Was here for lunch. Had ordered a pork BBQ set...
1,5 stars,I will give 10 stars if permitted! Food is gre...
2,5 stars,I just keep returning. ❤️🌟This is absolutely m...
3,5 stars,"Excellent service, reasonably priced set meal ..."
4,5 stars,Throwback to one of the better Korean BBQs I h...
...,...,...
177514,5 stars,(Translated by Google) The sea breeze blows co...
177515,4 stars,"(Translated by Google) Not bad, very comfortab..."
177516,5 stars,One of the earliest specialize in White Bee Ho...
177517,5 stars,White beehoon is the signature. You don't eat ...


In [ ]:
df['Labels'] = df['Review Rate'].str.replace('star', '')
df['Labels'] = df['Labels'].str.replace('s', '')
df['Labels'] = df['Labels'].str.strip()
df['Labels'] = df['Labels'].astype(int)
df['Labels'] = df['Labels'].map(lambda x: x-1)
df = df.drop(["Review Rate"], axis=1)
df['Review Text'] = df["Review Text"].str.split('\(Original\)').str[0]
df['Review Text'] = df["Review Text"].str.replace('\n', ' ')
df['Review Text'] = df["Review Text"].str.replace('\(Translated by Google\)', '')
df['Review Text'] = df["Review Text"].str.strip()
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


,Review Text,Labels
0,Was here for lunch. Had ordered a pork BBQ set...,4
1,I will give 10 stars if permitted! Food is gre...,4
2,I just keep returning. ❤️🌟This is absolutely m...,4
3,"Excellent service, reasonably priced set meal ...",4
4,Throwback to one of the better Korean BBQs I h...,4
...,...,...
177514,"The sea breeze blows cool, the park pavilion c...",4
177515,"Not bad, very comfortable environment, the foo...",3
177516,One of the earliest specialize in White Bee Ho...,4
177517,White beehoon is the signature. You don't eat ...,4


In [ ]:
df.tail()

,Review Text,Labels
177514,"The sea breeze blows cool, the park pavilion c...",4
177515,"Not bad, very comfortable environment, the foo...",3
177516,One of the earliest specialize in White Bee Ho...,4
177517,White beehoon is the signature. You don't eat ...,4
177518,"Like the mala bee hoon, the homemade tauhu als...",3


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 30.1 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
     |████████████████████████████████| 596 kB 59.5 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, MaxPool1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda
from tensorflow.keras.layers import Dense, Input, Flatten,SpatialDropout1D

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
token = tokenizer.encode_plus(
    df['Review Text'].iloc[0], 
    max_length=30, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [ ]:
token.input_ids

<tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[  101,  3982,  1303,  1111,  5953,   119,  6467,  2802,   170,
        19915,   139,  2064,  4880,  1383,   117,  2502,  1114,  1714,
         5543,  9069,  1105,   170,  2343, 24263, 26036,   119, 24930,
         4902,  1113,   102]], dtype=int32)>

In [ ]:
X_input_ids = np.zeros((len(df), 30))
X_attn_masks = np.zeros((len(df), 30))

In [ ]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['Review Text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=30, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [ ]:
labels = np.zeros((len(df), 5))
labels.shape

(177519, 5)

In [ ]:
labels[np.arange(len(df)), df['Labels'].values] = 1 # one-hot encoded target tensor

In [ ]:
labels

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.]])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(30,), dtype=tf.float64, name=None), TensorSpec(shape=(30,), dtype=tf.float64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset

In [ ]:

dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(30,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(30,), dtype=tf.float64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 30), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 30), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [ ]:
p = 0.9
train_size = int((len(df)//16)*p)

In [ ]:
train_size

9984

In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
from transformers import TFBertModel

In [ ]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(CustomLayer, self).__init__()
        self.model = TFBertModel.from_pretrained('bert-base-cased')
        

    def get_config(self):
        return {'BertLayer': self.model}

    def call(self, input):
        return tf.multiply(input, 2)

In [ ]:
input_ids = tf.keras.layers.Input(shape=(30,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(30,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[0] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
dropout = tf.keras.layers.Dropout(0.2, name="dropout")(intermediate_layer)
output_layer = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(dropout) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 30)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 30,                                            

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 2,
                                                      restore_best_weights = True)

In [ ]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=6,
    batch_size=16,
    callbacks = [earlystop_callback]
)

Epoch 1/6
9984/9984 [==============================] - 1194s 119ms/step - loss: 0.8387 - accuracy: 0.6373 - val_loss: 0.8177 - val_accuracy: 0.6468
Epoch 2/6
9984/9984 [==============================] - 1188s 119ms/step - loss: 0.7431 - accuracy: 0.6769 - val_loss: 0.7735 - val_accuracy: 0.6700
Epoch 3/6
9984/9984 [==============================] - 1186s 119ms/step - loss: 0.6656 - accuracy: 0.7116 - val_loss: 0.7532 - val_accuracy: 0.6869
Epoch 4/6
9984/9984 [==============================] - 1185s 119ms/step - loss: 0.5833 - accuracy: 0.7510 - val_loss: 0.7157 - val_accuracy: 0.7139
Epoch 5/6
9984/9984 [==============================] - 1184s 119ms/step - loss: 0.5028 - accuracy: 0.7880 - val_loss: 0.7053 - val_accuracy: 0.7217
Epoch 6/6
9984/9984 [==============================] - 1184s 119ms/step - loss: 0.4318 - accuracy: 0.8194 - val_loss: 0.7075 - val_accuracy: 0.7327


In [ ]:
sentiment_model.save('sentiment_model.h5')

In [ ]:
new_sentiment_model = tf.keras.models.load_model('sentiment_model.h5')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=30, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Negative', 'A bit negative', 'Neutral', 'A bit positive', 'Positive']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=30, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=[0,1,2,3,4]):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
!zip -r /content/model_v3.zip /content/sentiment_model_v3

  adding: content/sentiment_model_v3/ (stored 0%)
  adding: content/sentiment_model_v3/keras_metadata.pb (deflated 95%)
  adding: content/sentiment_model_v3/variables/ (stored 0%)
  adding: content/sentiment_model_v3/variables/variables.data-00000-of-00001 (deflated 15%)
  adding: content/sentiment_model_v3/variables/variables.index (deflated 80%)
  adding: content/sentiment_model_v3/assets/ (stored 0%)
  adding: content/sentiment_model_v3/saved_model.pb (deflated 92%)


In [ ]:
input_text = input('Enter review here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(new_sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")

Enter review here: not so good service prefer other restaurants
Predicted Sentiment: 0
